In [21]:
import pandas as pd
import os
import datetime as dt
import re
import xlsxwriter
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px

In [22]:
def get_right_date_format(x):
    """
    Преобразует даты, написанные некорректно
    """
    if x is None:
        return None
    if isinstance(x, dt.datetime):
        return x
    if isinstance(x, float):
        return x
    if x[2] == '.':
        x = x[-4:] + "-" + x[3:5] + "-" + x[0:2]
    if x == "2020-09-31":
        x = "2020-09-30"
    if x.isalpha():
        return None
    return x

def fix_telephone(row):
    """
    Приводит номера телефонов к единому формату
    """
    starts_with_nine = re.compile("9\d{9}")
    starts_with_seven = re.compile("7\d{10}")
    starts_with_eight = re.compile("8\d{10}")
    row = str(row).strip()
    unwanted_symbols = [' ', '  ', '   ', '+', '-', '(', ')']
    for i in unwanted_symbols:
        row = str(row).replace(i, '')
    if starts_with_nine.match(row):
        fixed_telephone = "+7(" + row[0:3] +")" + row[3:6] + "-" + row[6:8] + "-" + row[8:10]
    elif starts_with_seven.match(row) or starts_with_eight.match(row):
        fixed_telephone = "+7(" + row[1:4] +")" + row[4:7] + "-" + row[7:9] + "-" + row[9:11]
    else:
#         print(f"wrong number: {row}")
        fixed_telephone = "Invalid number:" + row
    return fixed_telephone

def get_this_monday():
    """Находит дату последнего понеделтника"""
    today = dt.date.today()
    this_monday = today - dt.timedelta(days=today.weekday())
    return this_monday

def get_last_week():
    """Возвращает список дат предыдущей недели"""
    this_monday = get_this_monday()
    last_monday = this_monday - dt.timedelta(days=7)
    last_week = pd.date_range(last_monday, periods=7, freq="D")
    print("Periuod:", last_week)
    return last_week
    
def get_two_weeks_bef_last_week():
    """Возвращает список дат двух недель перед предыдущей неделей"""
    this_monday = get_this_monday()
    two_mondays_bef_last_week = this_monday - dt.timedelta(days=21)
    two_weeks_bef_last_week = pd.date_range(two_mondays_bef_last_week, periods=14, freq="D")
    print("2 weeks before:", two_weeks_bef_last_week)
    return two_weeks_bef_last_week

In [23]:
class ReportFromOP():
    """
    Класс предназначен для обработки еженедельного отчета HR департамента по направленным на собеседование кандидатам
    op_rep_path, div_lst_path - путь до файлов
    .with_divs - отчет по направленным с добавлением индикатора о регионе и дивизионе, а так же
    с корректными датами и форматами телефонов
    .pivot - сводная таблица по отчету, с с добавлением новых столбцов и доп. фильтров
    """
    def __init__(self, op_rep_path, div_lst_path):
        self.op_rep_path = op_rep_path
        self.div_lst_path = div_lst_path
        self.with_divs, self.divs_lst = self.load_reports()
        self.pivot = self.get_pivot_tables()
    
    def load_reports(self):
        """
        Загружает отчет по направленным и список ресторанов с девизионами и регионами
        """
        report_from_op = pd.read_excel(self.op_rep_path, sheet_name="Лист1", parse_dates=False, engine='openpyxl',
                                       keep_default_na=True, convert_float=True, dtype={"Ресторан № (собеседование)": str,
                                                                                "Ресторан № (работа)": str, 
                                                                                        "ТелефонвформатеОРИС": str}, na_values=[pd.NaT])
        #     Приводим неправильно заполненные даты в машиночитаемый формат:
        report_from_op['Дата собеседования'] = pd.to_datetime(report_from_op['Дата собеседования'].apply(get_right_date_format))
        report_from_op['Телефонное интервью (число)'] = pd.to_datetime(report_from_op['Телефонное интервью (число)'].apply(get_right_date_format))
        report_from_op['ТелефонвформатеОРИС'] = report_from_op["ТелефонвформатеОРИС"].apply(fix_telephone)
        #     Переводим в нижний регистр для сокращения количества вариантов написания 
        report_from_op['Дошёл/Не дошёл ДА/НЕТ'] = report_from_op['Дошёл/Не дошёл ДА/НЕТ'].str.lower()
        report_from_op['Прошел МО ДА/НЕТ'] = report_from_op['Прошел МО ДА/НЕТ'].str.lower()
        report_from_op['Обратная связь от ресторана'] = report_from_op['Обратная связь от ресторана'].str.lower()
        report_from_op['Обратная связь от кандидата'] = report_from_op['Обратная связь от кандидата'].str.lower()
        # Загружаем список дивизионов, регионов и ресторанов
        divizion_lst = pd.read_excel(self.div_lst_path, sheet_name="Лист1", parse_dates=True, engine='openpyxl', 
                               keep_default_na=True, convert_float=True, dtype={'код ресторана': str},
                              na_values=[pd.NaT])
        # Добавляем поментку о дивизионе и регионе в отчет
        merged_table = pd.merge(report_from_op, divizion_lst, left_on='Ресторан № (работа)', right_on='код ресторана')
        return merged_table, divizion_lst   
    
    def merge_pivot_tables(self, pivot_napravleno_ot_op, pivot_came_from_op, pivot_no_os, pivot_passed_mo, 
                          pivot_napravleno_this_week):
        """
        Склеивает полученные после фильтрации и аггрегирования сводные таблицы в итоговую
        """
        # Создаем финальную таблицу с кодами ресторанов
        final_table = self.divs_lst[['код ресторана', 'Дивизион', 'регион']].copy()

        # Словарь с названиями сводных таблиц
        pivot_dict = {'Направленные от ОП': pivot_napravleno_ot_op, 
                      'Пришли от ОП': pivot_came_from_op, 
                      'Не дали ОС':pivot_no_os, 
                      'Прошли МО': pivot_passed_mo, 
                      'Направлено за текущ. нед.': pivot_napravleno_this_week}

        # Добавляем в цикле колонки с получившимися значениями в финальную таблицу
        for (i, pivot_table) in pivot_dict.items():
            final_table = pd.merge(final_table, pivot_table, left_on='код ресторана', how='outer', right_on='Ресторан № (работа)')
            print(f"Column {i} added to final_table")
        return final_table
        
    
    def get_pivot_tables(self):
        """
        Фильтруе данные и создает сводные таблицы
        """
        # Уникальные значения, входящие в "Да" (для фильтрации столбцов 'Пришли от ОП' и 'Прошли МО')
        yes_lst = ['да', ' ',  '  ', ' да', 'да ', 'дошла', 'дошел']
        last_week = get_last_week()
        two_weeks_bef_last_week = get_two_weeks_bef_last_week()

        #  Вычисляем 'Направленные от ОП' - дата собесед, кот. входит в исследуемую неделю
        napravleno_ot_op = self.with_divs[self.with_divs['Дата собеседования'].isin(last_week)]
        pivot_napravleno_ot_op = pd.pivot_table(napravleno_ot_op, values='ФИО', index='Ресторан № (работа)', aggfunc='count')
        pivot_napravleno_ot_op = pivot_napravleno_ot_op.rename(columns={'ФИО':'Направленные от ОП'})
        print(pivot_napravleno_ot_op.sum())

        #  Вычисляем 'Пришли от ОП' - дошел/не дошел выбираем только да и пустые в исследуемую неделю
        print("Уникальные значения столбца 'Дошёл/Не дошёл ДА/НЕТ':", self.with_divs['Дошёл/Не дошёл ДА/НЕТ'] .unique())
        came_from_op = napravleno_ot_op[napravleno_ot_op['Дошёл/Не дошёл ДА/НЕТ'].isin(yes_lst)]
        pivot_came_from_op = pd.pivot_table(came_from_op, values='ФИО', index='Ресторан № (работа)', aggfunc='count')
        pivot_came_from_op = pivot_came_from_op.rename(columns={'ФИО':'Пришли от ОП'})
        print(pivot_came_from_op.sum())

        # Вычисляем 'Не дали ОС' - дата собесед. входит в исследуемую неделю + 'Обратная связь от ресторана' = NaN
        no_os = napravleno_ot_op[pd.isna(napravleno_ot_op['Обратная связь от ресторана'])]
        pivot_no_os = pd.pivot_table(no_os, values='ФИО', index='Ресторан № (работа)', aggfunc='count')
        pivot_no_os = pivot_no_os.rename(columns={'ФИО':'Не дали ОС'})
        print(pivot_no_os.sum())

        # 'Прошли МО' - дата собеседования - предыдущие две недели до исследуемой недели + прошел МО "Да"
        print("Уникальные значения столбца 'Прошел МО ДА/НЕТ':", self.with_divs['Прошел МО ДА/НЕТ'].unique())
        napravleno_ot_op_two_weeks_ago = self.with_divs[self.with_divs['Дата собеседования'].isin(two_weeks_bef_last_week)]
        passed_mo = napravleno_ot_op_two_weeks_ago[napravleno_ot_op_two_weeks_ago['Прошел МО ДА/НЕТ'].isin(yes_lst)]
        pivot_passed_mo = pd.pivot_table(passed_mo, values='ФИО', index='Ресторан № (работа)', aggfunc='count')
        pivot_passed_mo = pivot_passed_mo.rename(columns={'ФИО':'Прошли МО'})
        print(pivot_passed_mo.sum())

        # Вычисляем 'Направлено за текущ. нед.' - дата телефонного интервью входит в исследуемую неделю
        napravleno_this_week = self.with_divs[self.with_divs['Телефонное интервью (число)'].isin(last_week)]
        pivot_napravleno_this_week = pd.pivot_table(napravleno_this_week, values='ФИО', index='Ресторан № (работа)', aggfunc='count')
        pivot_napravleno_this_week = pivot_napravleno_this_week.rename(columns={'ФИО':'Направлено за текущ. нед.'})
        print(pivot_napravleno_this_week.sum())
        
        # Склеиваем все столбцы в одну таблицу
        merged_pivot = self.merge_pivot_tables(pivot_napravleno_ot_op, pivot_came_from_op, pivot_no_os,
                                          pivot_passed_mo, pivot_napravleno_this_week)
        return merged_pivot
    
    
    def save_final_report(self):
        """
        Сохраняет таблицу в Excel файл
        """
        file_dates = get_last_week()
        file_name = f"Сводная направленные от ОП {file_dates[0].day}.{file_dates[0].month}.{file_dates[0].year}-{file_dates[-1].day}.{file_dates[-1].month}.{file_dates[-1].year}.xlsx"
        print(file_name)
        with pd.ExcelWriter(file_name, 
                            engine='xlsxwriter',
                            datetime_format='dd.mm.yyyy') as writer:
            self.with_divs.to_excel(writer, sheet_name='Данные', index=False)
            self.pivot.to_excel(writer, sheet_name='Сводная', index=False)
            print(f"Excel file {file_name} was created")
            

In [24]:
class DivisionReport():
    """
    Разделяет отчет по направленным по дивизионам, оставляя только данные, отноящиеся к тому или иному дивизиону
    division_name - название дивизиона ("Евразия" / "ММО" / "Центральная Россия")
    """
    def __init__(self, report, division_name):
        self.report = report
        self.division_name = division_name
        self.div_report = self.get_division_report()
        
    def get_division_report(self):
        """
        Оставляет в отчете только информацию по девизиону
        """
        self.report = self.report.drop(columns=['Unnamed: 10', 'код ресторана', 'город'])
        division_report = self.report[self.report['Дивизион'] == self.division_name]
        division_report = division_report.sort_values(by='Дата собеседования')
        return division_report

    def get_file_name(self):
        """
        Склеивает дату отчета и название файла
        """
        last_week = get_last_week()
        file_name = f"{self.division_name} Направленные кандидаты от ОП от 25.05.20-{last_week[-1].day}.{last_week[-1].month}.{last_week[-1].year}.xlsx"
        return file_name

    def save_div_report(self):
            """
            Сохраняет таблицу в Excel файл
            """
            file_name = self.get_file_name()
            print(file_name)
            with pd.ExcelWriter(file_name, 
                                engine='xlsxwriter',
                                datetime_format='dd.mm.yyyy') as writer:
                self.report.to_excel(writer, sheet_name='Данные', index=False)
                print(f"Excel file {file_name} was created")

In [25]:
op_report = ReportFromOP('Направленные кандидаты от ОП от 25.05.-27.12.2020.xlsx', 'распределение ресоранов по девизионам.xlsx')
# op_report.with_divs


C:\Users\ekaterina.riabenko\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning:

Data Validation extension is not supported and will be removed



Periuod: DatetimeIndex(['2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07',
               '2021-01-08', '2021-01-09', '2021-01-10'],
              dtype='datetime64[ns]', freq='D')
2 weeks before: DatetimeIndex(['2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24',
               '2020-12-25', '2020-12-26', '2020-12-27', '2020-12-28',
               '2020-12-29', '2020-12-30', '2020-12-31', '2021-01-01',
               '2021-01-02', '2021-01-03'],
              dtype='datetime64[ns]', freq='D')
Series([], dtype: float64)
Уникальные значения столбца 'Дошёл/Не дошёл ДА/НЕТ': ['нет' 'да' nan None 'нет ' 'да ' ' ' '  ' ' да']
Series([], dtype: float64)
Series([], dtype: float64)
Уникальные значения столбца 'Прошел МО ДА/НЕТ': [None nan 'нет' 'да' 'нeт' 'нет ' 'да ']
Прошли МО    11
dtype: int64
Series([], dtype: float64)
Column Направленные от ОП added to final_table
Column Пришли от ОП added to final_table
Column Не дали ОС added to final_table
Column Прошли МО added to final_table

In [26]:
# Сохратнит отчет в ексель
op_report.save_final_report()

Periuod: DatetimeIndex(['2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07',
               '2021-01-08', '2021-01-09', '2021-01-10'],
              dtype='datetime64[ns]', freq='D')
Сводная направленные от ОП 4.1.2021-10.1.2021.xlsx
Excel file Сводная направленные от ОП 4.1.2021-10.1.2021.xlsx was created


In [8]:
eurasia = DivisionReport(op_report.with_divs, 'Евразия')
mmo = DivisionReport(op_report.with_divs, 'ММО')
center = DivisionReport(op_report.with_divs, 'Центральная Россия')

In [9]:
# # Сохранение отчетов по девизионам в Excel
eurasia.save_div_report()
mmo.save_div_report()
center.save_div_report()